<a href="https://colab.research.google.com/github/Har-jas-Kaur/Gesture-to-text/blob/main/Gesturetotext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Sep 24 09:31:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import cv2
import numpy as np

from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers,models


In [ ]:
import cv2
import numpy as np
from tqdm import tqdm
import string

# Define the paths to your train and test data folders
train_data_dir = '/content/drive/MyDrive/sign_lang/data1/train'
test_data_dir = '/content/drive/MyDrive/sign_lang/data1/test'

# Define the target image size (e.g., 224x224 pixels)
image_height, image_width = 224, 224

# Initialize empty lists to store preprocessed data
X_train = []
X_test = []
Y_train = []
Y_test = []

class_label={}
for char in string.ascii_uppercase:
  class_label[char]=ord(char)-ord('A')

j=26
for i in range(0,10):
  class_label[str(i)]=j
  j=j+1

print(class_label)

# Loop through the training data folders
for class_name, label in class_label.items():
    class_dir_train = os.path.join(train_data_dir, class_name)
    class_dir_test = os.path.join(test_data_dir, class_name)

    # Loop through the images in the training folder
    for img_name in tqdm(os.listdir(class_dir_train)):
        img_path = os.path.join(class_dir_train, img_name)

        # Read and resize the image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (image_width, image_height))

        # Normalize the pixel values (scaling to [0, 1])
        img = img.astype(np.float32) / 255.0

        # Append the preprocessed image to the training data
        X_train.append(img)
        Y_train.append(label)

    # Loop through the images in the test folder
    for img_name in tqdm(os.listdir(class_dir_test)):
        img_path = os.path.join(class_dir_test, img_name)

        # Read and resize the image
        img = cv2.imread(img_path)
        img = cv2.resize(img, (image_width, image_height))

        # Normalize the pixel values (scaling to [0, 1])
        img = img.astype(np.float32) / 255.0

        # Append the preprocessed image to the test data
        X_test.append(img)
        Y_test.append(label)

# Convert the lists to NumPy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
y_test = np.array(Y_test)

# Shuffle the data (optional)
shuffle_indices = np.arange(X_train.shape[0])
np.random.shuffle(shuffle_indices)
X_train = X_train[shuffle_indices]
Y_train = Y_train[shuffle_indices]

# Print the shape of your preprocessed data to ensure correctness
print("X_train shape:", X_train.shape)
print("y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, '0': 26, '1': 27, '2': 28, '3': 29, '4': 30, '5': 31, '6': 32, '7': 33, '8': 34, '9': 35}


100%|██████████| 23/23 [00:17<00:00,  1.28it/s]


X_train shape: (1802, 224, 224, 3)
y_train shape: (1802,)
X_test shape: (720, 224, 224, 3)
y_test shape: (720,)


In [ ]:
print(type(X_train))
print(type(Y_train))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(class_label), activation='softmax')  # Output layer with class count
])

model.save("langdet.keras")

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=7, batch_size=32, validation_split=0.2)


Epoch 1/7
46/46 [==============================] - 16s 75ms/step - loss: 3.0145 - accuracy: 0.2575 - val_loss: 1.1875 - val_accuracy: 0.6704
Epoch 2/7
46/46 [==============================] - 3s 58ms/step - loss: 0.3919 - accuracy: 0.9042 - val_loss: 0.4494 - val_accuracy: 0.9307
Epoch 3/7
46/46 [==============================] - 3s 60ms/step - loss: 0.3146 - accuracy: 0.9403 - val_loss: 0.1684 - val_accuracy: 0.9335
Epoch 4/7
46/46 [==============================] - 3s 56ms/step - loss: 0.1861 - accuracy: 0.9604 - val_loss: 0.0443 - val_accuracy: 0.9861
Epoch 5/7
46/46 [==============================] - 2s 53ms/step - loss: 0.0050 - accuracy: 0.9993 - val_loss: 0.0368 - val_accuracy: 0.9917
Epoch 6/7
46/46 [==============================] - 3s 55ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.0215 - val_accuracy: 0.9945
Epoch 7/7
46/46 [==============================] - 2s 53ms/step - loss: 3.0719e-04 - accuracy: 1.0000 - val_loss: 0.0290 - val_accuracy: 0.9945


In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

Cloning into 'darknet'...
remote: Enumerating objects: 15769, done.
remote: Counting objects: 100% (231/231), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 15769 (delta 118), reused 162 (delta 89), pack-reused 15538
Receiving objects: 100% (15769/15769), 14.62 MiB | 1.98 MiB/s, done.
Resolving deltas: 100% (10552/10552), done.


In [ ]:
# Change directory to the YOLOv4 repository
%cd darknet

# Compile YOLOv4 (this may take some time)
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1444:14: warning: unused variable ‘bu

In [ ]:
import cv2

# Load YOLOv4 or another YOLO-based object detection model
yolo_net = cv2.dnn.readNet('yolov4.cfg', 'yolov4.weights')

# Load your pre-trained gesture recognition model
gesture_model = tf.keras.models.load_model("langdet.keras")

# Open the webcam
cap = cv2.VideoCapture(0)

# Specify the target size for resizing input frames
target_size = (224, 224)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Resize the input frame to match the size used during training
    resized_frame = cv2.resize(frame, target_size)

    # Perform object detection using YOLOv4
    blob = cv2.dnn.blobFromImage(resized_frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    yolo_net.setInput(blob)
    layer_names = yolo_net.getUnconnectedOutLayersNames()
    detections = yolo_net.forward(layer_names)

    # Process YOLO detections and perform gesture recognition
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:  # You can adjust the confidence threshold
                # Extract bounding box coordinates
                center_x = int(obj[0] * resized_frame.shape[1])
                center_y = int(obj[1] * resized_frame.shape[0])
                width = int(obj[2] * resized_frame.shape[1])
                height = int(obj[3] * resized_frame.shape[0])
                x = center_x - width // 2
                y = center_y - height // 2

                detected_object = resized_frame[y:y + height, x:x + width]

                # Perform gesture recognition on the detected object using your custom model
                # Apply preprocessing as needed for your model
                preprocessed_object = detected_object / 255.0  # Example scaling to [0, 1]

                predictions = model.predict(preprocessed_object)  # Output shape might be (1, num_classes)
                gesture_label = np.argmax(predictions, axis=1)


                #gesture_label = recognize_gesture(preprocessed_object, gesture_model)  # You should implement this function

                # Draw bounding box and gesture label on the original-sized frame
                x *= frame.shape[1] // target_size[0]
                y *= frame.shape[0] // target_size[1]
                width *= frame.shape[1] // target_size[0]
                height *= frame.shape[0] // target_size[1]

                cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
                cv2.putText(frame, gesture_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with detected objects and recognized gestures
    cv2.imshow('Gesture Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: ignored